In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [19]:
url = 'https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(21)00048-7/fulltext'
response = requests.get(url)
# print(response.text)

In [29]:
soup = BeautifulSoup(response.text, 'lxml')
table = soup.find_all('tbody')[0].find_all('td')

In [30]:
table

[<td>Abbott BinaxNOW, USA</td>,
 <td>Nasal swab</td>,
 <td>0–7 days</td>,
 <td>Visual, 15 min</td>,
 <td>97%, 99%</td>,
 <td>WHO Emergency Use Listing; US FDA Emergency Use Authorization; app for results; influenza A and B tests available</td>,
 <td>Abbott Panbio, USA</td>,
 <td>Nasal swab, nasopharyngeal swab</td>,
 <td>0–7 days</td>,
 <td>Visual, 15–20min</td>,
 <td>93%, 99%</td>,
 <td>WHO Emergency Use Listing; US FDA Emergency Use Authorization pending</td>,
 <td>Access Bio CareStart, USA</td>,
 <td>Nasal swab, nasopharyngeal swab</td>,
 <td>0–5 days</td>,
 <td>Visual, 15–20min</td>,
 <td>88%, 100%</td>,
 <td>US FDA Emergency Use Authorization</td>,
 <td>BD Veritor, USA</td>,
 <td>Nasal swab</td>,
 <td>0–5 days</td>,
 <td>Instrument, 30 min</td>,
 <td>84%, 100%</td>,
 <td>US FDA Emergency Use Authorization</td>,
 <td>LumiraDx, UK</td>,
 <td>Nasal swab</td>,
 <td>0–12 days</td>,
 <td>Instrument, 12 min</td>,
 <td>98%, 97%</td>,
 <td>US FDA Emergency Use Authorization</td>,
 <td>Quid

In [32]:
contents = [value.text for value in table]
contents

['Abbott BinaxNOW, USA',
 'Nasal swab',
 '0–7 days',
 'Visual, 15 min',
 '97%, 99%',
 'WHO Emergency Use Listing; US FDA Emergency Use Authorization; app for results; influenza A and B tests available',
 'Abbott Panbio, USA',
 'Nasal swab, nasopharyngeal swab',
 '0–7 days',
 'Visual, 15–20min',
 '93%, 99%',
 'WHO Emergency Use Listing; US FDA Emergency Use Authorization pending',
 'Access Bio CareStart, USA',
 'Nasal swab, nasopharyngeal swab',
 '0–5 days',
 'Visual, 15–20min',
 '88%, 100%',
 'US FDA Emergency Use Authorization',
 'BD Veritor, USA',
 'Nasal swab',
 '0–5 days',
 'Instrument, 30 min',
 '84%, 100%',
 'US FDA Emergency Use Authorization',
 'LumiraDx, UK',
 'Nasal swab',
 '0–12 days',
 'Instrument, 12 min',
 '98%, 97%',
 'US FDA Emergency Use Authorization',
 'Quidel Sofia SARS Antigen Fluorescent Immunoassay, USA',
 'Nasal swab, nasopharyngeal swab',
 '0–5 days',
 'Instrument, 20 min',
 '97%, 100%',
 'US FDA Emergency Use Authorization; does not differentiate between SARS-

In [50]:
tests = [text for i, text in enumerate(contents) if i%6 == 0]
sensitivities = [text.split(', ')[0] for i, text in enumerate(contents) if i%6 == 4]
specificities = [text.split(', ')[1] for i, text in enumerate(contents) if i%6 == 4]

In [51]:
df = pd.DataFrame({'Test_Name':tests, 'Sensitivity':sensitivities, 'Specificity':specificities})
df['Sensitivity'] = df['Sensitivity'].map(lambda x: int(x.strip('%')))/100
df['Specificity'] = df['Specificity'].map(lambda x: int(x.strip('%')))/100
df

,Test_Name,Sensitivity,Specificity
0,"Abbott BinaxNOW, USA",0.97,0.99
1,"Abbott Panbio, USA",0.93,0.99
2,"Access Bio CareStart, USA",0.88,1.00
3,"BD Veritor, USA",0.84,1.00
4,"LumiraDx, UK",0.98,0.97
5,Quidel Sofia SARS Antigen Fluorescent Immunoas...,0.97,1.00
6,Quidel Sofia Flu and SARS Antigen Fluorescent ...,0.95,1.00
7,"SD Biosensor, South Korea",0.97,1.00


In [52]:
def bayes(pBA, pA, pBnA):
    # takes in:
    # probability of B given A (pBA)
    # probability of A (pA)
    # probability of B given not A (pBnA)
    # returns: probability of A given B
    
    return pBA*pA/(pBA*pA + pBnA*(1-pA))

In [57]:
df['P(virus|+)'] = bayes(df['Sensitivity'], 0.17, 1-df['Specificity'])
df['P(virus|-)'] = bayes(1-df['Sensitivity'], 0.17, df['Specificity'])